In [7]:

#dog breed indentifier using CNN
# Importing the libraries
import tensorflow as tf


# gpu_devices = tf.config.experimental.list_physical_devices('GPU')
# print(gpu_devices)

# for device in gpu_devices:
#     tf.config.experimental.set_memory_growth(device, True)

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# This disables the GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except e:
    # Invalid device or cannot modify virtual devices once initialized.
    print(e)
    pass

import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import os
import cv2
import random
import pickle
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
# from tensorflow.keras.callbacks import TensorBoard
import time
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
# from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import categorical_crossentropy


In [8]:
RESULTS_DIR = './results/'
IMG_SIZE = 224

BATCH_SIZE_PRE = 32
BATCH_SIZE_FINE = 32
EPOCHS_PRE = 5
EPOCHS_FINE = 12
MOMENTUM = 0.9

In [9]:
def read_image4d_from_file(filename:str):
    image = cv2.imread(filename, cv2.IMREAD_COLOR).astype('float32')
    print("image type:", type(image))
    image_4d = np.expand_dims(image, axis=0)
    print("image_4d:", image_4d.shape)
    return image_4d

In [10]:
# Heavily inspired by Xception-with-Your-Own-Dataset implementation of 
def generator_from_filenames_and_labels(filenames, labels, batch_size, input_size=(299,299)):
    num_files = len(filenames)
    while 1:
        permutation = np.random.permutation(num_files)
        inputs_shuffled = filenames[permutation]
        labels_shuffled = labels[permutation]
        for i in range(0, numsamples, batch_size):
            inputs = list(map(lambda x: image.load_img(x, target_size=input_size), filenames[i:i+batch_size]))
            inputs = np.array(list(map(lambda x: image.img_to_array(x))))
            inputs = pre_process_inputs(inputs)
            yield (inputs, labels[i:i+batch_size])

In [11]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
#     featurewise_center=True,
#     featurewise_std_normalization=True,
    rotation_range = 10,
    horizontal_flip = True,
    validation_split = 0.2)

train_pre_generator = train_datagen.flow_from_directory(
    directory = "./data/fusion/train/",
    target_size = (IMG_SIZE, IMG_SIZE),
    color_mode = "rgb",
    batch_size = BATCH_SIZE_PRE,
    class_mode = "categorical")

train_fine_generator = train_datagen.flow_from_directory(
    directory = "./data/fusion/train/",
    target_size = (IMG_SIZE, IMG_SIZE),
    color_mode = "rgb",
    batch_size = BATCH_SIZE_FINE,
    class_mode = "categorical")

test_datagen = ImageDataGenerator(
    rescale=1.0/255
)
test_generator = test_datagen.flow_from_directory(
    directory = "./data/fusion/test/",
    batch_size = BATCH_SIZE_FINE,
    class_mode = "categorical"
)


Found 18680 images belonging to 172 classes.
Found 18680 images belonging to 172 classes.
Found 9355 images belonging to 172 classes.


In [12]:
# # Importing the dataset
# dog_breed_name = pd.read_csv('./data/labels_original.csv')
# dog_breed_name.head()

In [13]:
# # find files in the directory
# images_train = os.listdir("./data/fusion/train/")
# # import test data set
# image_test = os.listdir("./data/fusion/test/")

# # from https://stackoverflow.com/questions/2632205/how-to-count-the-number-of-files-in-a-directory-using-python
# num_images_train = len([name for name in os.listdir('./data/fusion/train/') if os.path.isfile(name)])
# num_images_test = len([name for name in os.listdir('./data/fusion/test/') if os.path.isfile(name)])

# print(num_images_train)
# print(num_images_test)
# print(len(images_train))

In [14]:
# # create target variable
# dog_breed_name['target'] = dog_breed_name['breed'].astype('category').cat.codes
# dog_breed_name.head()

In [15]:
# dog_breed_dict = {}
# for i, breed in enumerate(dog_breed_name['breed'].unique()):
#     dog_breed_dict[breed] = i
    
# num_breeds = len(dog_breed_name['breed'].unique())
# print("num_breeds:", num_breeds)

assert len(os.listdir('./data/fusion/train/')) == len(os.listdir('./data/fusion/test/'))
num_breeds = len(os.listdir('./data/fusion/train/')) - 1
print("num_breeds:", num_breeds)

num_breeds: 172


In [16]:
mobilenetv2_base_model = mobilenet_v2.MobileNetV2(include_top=False,
                                        weights='imagenet',
                                        input_shape=(IMG_SIZE,IMG_SIZE,3))

x_base_mobilenetv2 = mobilenetv2_base_model.output
x_avg_pooling_mobilenetv2 = GlobalAveragePooling2D()(x_base_mobilenetv2)
x_first_fc_mobilenetv2 = Dense(1024, activation='relu')(x_avg_pooling_mobilenetv2)
x_last_fc_mobilenetv2 = Dense(num_breeds, activation='softmax')(x_first_fc_mobilenetv2)

mobilenetv2_custom_model = Model(inputs=mobilenetv2_base_model.inputs, outputs=x_last_fc_mobilenetv2)

# xception_custom_model.summary()

2022-12-13 00:28:33.501845: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
#Train model - freeze body layers first
for layer in mobilenetv2_base_model.layers:
    layer.trainable = False

mobilenetv2_custom_model.compile(optimizer=SGD(momentum=MOMENTUM), loss=categorical_crossentropy, metrics=['accuracy'])

model_train_pre = mobilenetv2_custom_model.fit_generator(
    generator = train_pre_generator,
    steps_per_epoch = (train_pre_generator.n//train_pre_generator.batch_size),
    epochs = EPOCHS_PRE,
    validation_data = test_generator,
    validation_steps = (test_generator.n//test_generator.batch_size),
    verbose=1
)

mobilenetv2_custom_model.save(os.path.join(RESULTS_DIR, 'model_pre_mbnet2_3.h5'))

/tmp/ipykernel_104902/2026603185.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_train_pre = mobilenetv2_custom_model.fit_generator(


Epoch 1/5
583/583 [==============================] - 253s 432ms/step - loss: 0.6016 - accuracy: 0.8108 - val_loss: 1.5169 - val_accuracy: 0.5879
Epoch 2/5
583/583 [==============================] - 245s 420ms/step - loss: 0.4843 - accuracy: 0.8453 - val_loss: 1.5118 - val_accuracy: 0.5954
Epoch 3/5
583/583 [==============================] - 249s 427ms/step - loss: 0.4296 - accuracy: 0.8636 - val_loss: 1.5063 - val_accuracy: 0.5926
Epoch 4/5
583/583 [==============================] - 250s 428ms/step - loss: 0.3974 - accuracy: 0.8726 - val_loss: 1.5267 - val_accuracy: 0.5917
Epoch 5/5
583/583 [==============================] - 251s 431ms/step - loss: 0.3609 - accuracy: 0.8829 - val_loss: 1.5543 - val_accuracy: 0.5890


In [17]:
#Train model - train all layers
for layer in mobilenetv2_base_model.layers:
    layer.trainable = True
    
mobilenetv2_custom_model.compile(optimizer=SGD(momentum=MOMENTUM), loss=categorical_crossentropy, metrics=['accuracy'] )

model_train_fine = mobilenetv2_custom_model.fit_generator(
    generator = train_fine_generator,
    steps_per_epoch = (train_fine_generator.n//train_fine_generator.batch_size),
    epochs = EPOCHS_FINE,
    validation_data = test_generator,
    validation_steps = (test_generator.n//test_generator.batch_size),
    verbose = 1
)

mobilenetv2_custom_model.save(os.path.join(RESULTS_DIR, 'model_fine_mbnet2.h5'))

Epoch 1/12


/tmp/ipykernel_104902/428998347.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_train_fine = mobilenetv2_custom_model.fit_generator(


1167/1167 [==============================] - 701s 599ms/step - loss: 2.5809 - accuracy: 0.3339 - val_loss: 12.4583 - val_accuracy: 0.0079
Epoch 2/12
1167/1167 [==============================] - 707s 606ms/step - loss: 1.9728 - accuracy: 0.4546 - val_loss: 6.5063 - val_accuracy: 0.0691
Epoch 3/12
1167/1167 [==============================] - 719s 616ms/step - loss: 1.6384 - accuracy: 0.5299 - val_loss: 3.9617 - val_accuracy: 0.1809
Epoch 4/12
1167/1167 [==============================] - 716s 614ms/step - loss: 1.4372 - accuracy: 0.5803 - val_loss: 5.4386 - val_accuracy: 0.0971
Epoch 5/12
1167/1167 [==============================] - 714s 612ms/step - loss: 1.2800 - accuracy: 0.6191 - val_loss: 2.8422 - val_accuracy: 0.3264
Epoch 6/12
1167/1167 [==============================] - 714s 612ms/step - loss: 1.1402 - accuracy: 0.6502 - val_loss: 3.3078 - val_accuracy: 0.2501
Epoch 7/12
1167/1167 [==============================] - 712s 610ms/step - loss: 1.0361 - accuracy: 0.6813 - val_loss: 2.30

In [ ]:
# def image_predict_from_model(filename:str):
#     image_4d = read_image_4d(filename)
#     preprocessed_image = preprocess_input(raw_image)
#     prediction_probs = xception_model.predict_generator(preprocessed_image)
#     best_prediction = np.argmax(prediction_probs)
    
# #     features = xception_models.predict(image)
# #     return features.reshape(1, 7*7*512)
    
    

In [19]:
# From Ananya, f-1 score
mobilenetv2_custom_model= load_model(os.path.join(RESULTS_DIR, 'model_pre_mbnet2_2.h5'))
y_pred = mobilenetv2_custom_model.predict_generator(test_generator, steps=test_generator.samples // BATCH_SIZE_FINE, verbose=1)

y_pred = np.argmax(y_pred, axis=1)

y_true = test_generator.classes

print('f1 score:', f1_score(y_true, y_pred, average='macro'))

/tmp/ipykernel_109829/2465302789.py:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  y_pred = mobilenetv2_custom_model.predict_generator(test_generator, steps=test_generator.samples // BATCH_SIZE_FINE, verbose=1)


292/292 [==============================] - 86s 293ms/step


NameError: name 'f1_score' is not defined